https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2023/05-deployment/homework.md

## Question 1

* Install Pipenv
* What's the version of pipenv you installed?
* Use `--version` to find out

In [ ]:
!pip install pipenv

In [1]:
!pipenv --version

pipenv, version 2023.10.3


## Question 2

* Use Pipenv to install Scikit-Learn version 1.3.1
* What's the first hash for scikit-learn you get in Pipfile.lock?

> **Note**: you should create an empty folder for homework
and do it there. 

In [ ]:
!pipenv install scikit-learn==1.3.1

In [ ]:
!pipenv install numpy flask

## Question 3

Let's use these models!

* Write a script for loading these models with pickle
* Score this client:

```json
{"job": "retired", "duration": 445, "poutcome": "success"}
```

What's the probability that this client will get a credit? 

* 0.162
* 0.392
* 0.652
* <mark>0.902</mark>

If you're getting errors when unpickling the files, check their checksum:

```bash
$ md5sum model1.bin dv.bin
8ebfdf20010cfc7f545c43e3b52fc8a1  model1.bin
924b496a89148b422c74a62dbc92a4fb  dv.bin
```

In [2]:
import pickle

In [3]:
# very important to use 'rb' here, it means read-binary 
with open('model1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

with open('dv.bin', 'rb') as f_in:
    dv = pickle.load(f_in)

In [4]:
model, dv

(LogisticRegression(), DictVectorizer(sparse=False))

In [5]:
customer = {
    "job": "retired", 
    "duration": 445, 
    "poutcome": "success"
}
X = dv.transform([customer])
y_pred = model.predict_proba(X)[0, 1]

y_pred, y_pred.round(3)

(0.9019309332297606, 0.902)

## Question 4

Now let's serve this model as a web service

* Install Flask and gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this client using `requests`:

```python
url = "YOUR_URL"
client = {"job": "unknown", "duration": 270, "poutcome": "failure"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a credit?

* <mark>0.140</mark>
* 0.440
* 0.645
* 0.845


## Docker

Install [Docker](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/05-deployment/06-docker.md). 
We will use it for the next two questions.

For these questions, we prepared a base image: `svizor/zoomcamp-model:3.10.12-slim`. 
You'll need to use it (see Question 5 for an example).

This image is based on `python:3.10.12-slim` and has a logistic regression model 
(a different one) as well a dictionary vectorizer inside. 

This is how the Dockerfile for this image looks like:

```docker 
FROM python:3.10.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
```

We already built it and then pushed it to [`svizor/zoomcamp-model:3.10.12-slim`](https://hub.docker.com/r/svizor/zoomcamp-model).

> **Note**: You don't need to build this docker image, it's just for your reference.

In [21]:
import requests

In [30]:
url = "http://127.0.0.1:9696/predict"
client = {"job": "unknown", "duration": 270, "poutcome": "failure"}
requests.post(url, json=client).json()

{'credit_probability': 0.13968947052356817, 'credit_probability_round': 0.14}

## Question 5

Download the base image `svizor/zoomcamp-model:3.10.12-slim`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 47 MB
* <mark>147 MB</mark>
* 374 MB
* 574 MB

You can get this information when running `docker images` - it'll be in the "SIZE" column.

## Question 6

Let's run your docker container!

After running it, score this client once again:

```python
url = "YOUR_URL"
client = {"job": "retired", "duration": 445, "poutcome": "success"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a credit now?

* 0.168
* 0.530
* 0.730
* <mark>0.968</mark>

In [32]:
url = "http://0.0.0.0:9697/predict"
client = {"job": "retired", "duration": 445, "poutcome": "success"}
requests.post(url, json=client).json()

{'credit_probability': 0.9019309332297606, 'credit_probability_round': 0.902}